In [ ]:
%pylab inline
from __future__ import print_function
from __future__ import division

# General audio work

In [ ]:
from scipy.io import wavfile

If scipy is not available, you can use the *wave* module which is part of the standard library:

http://docs.python.org/2/library/wave.html

However it is trickier to use as you get the bytes directly and must manually turn them into numbers.

    wavfile.write('outsig.wav', 44100, array(sigout, dtype=int16))

    sr, sample = wavfile.read('passport.wav')

Now we'll generate a bunch of files with different frequencies:

In [ ]:
freqs = [440, 660, 880, 1000, 2000]
dur = 2
sr = 44100

for freq in freqs:
    phs = linspace(0, 2 * pi * freq * dur, sr*dur)
    samps = sin(phs)* (2.0**15)
    wavfile.write('result/tone-%i.wav'%freq, sr, samps.astype(int16))

In [ ]:
ls

In [ ]:
ls result

In [ ]:
sr, sample = wavfile.read('result/tone-440.wav')
plot(sample[0:200])

In [ ]:
!afplay result/tone-440.wav

## Interleaving/De-interleaving

In [ ]:
dur = 2
sr = 44100

phs1 = linspace(0, 2 * pi * 440 * dur, sr*dur)
samps1 = sin(phs1)* (2.0**15)
phs2 = linspace(0, 2 * pi * 660 * dur, sr*dur)
samps2 = sin(phs2)* (2.0**15)

wavfile.write('result/tone_stereo.wav', sr, array((samps1, samps2)).T.astype(int16) )

In [ ]:
!afplay tone_stereo.wav

In [ ]:
sr, samples = wavfile.read('result/tone_stereo.wav')
samples.shape

In [ ]:
interleaved = samples.flat

In [ ]:
len(interleaved)

In [ ]:
nchnls = 2
sampindex = 20
print(samples[sampindex,:])
print(interleaved[nchnls * sampindex], interleaved[nchnls * sampindex + 1])

In [ ]:
chan1 = interleaved[::2]

In [ ]:
chan1.shape

In [ ]:
chan2 = interleaved[1::2]

## Scripting

Perform actions on a large number of files.

For example add a fadeout to all files in a directory:

In [ ]:
import glob

In [ ]:
glob.glob('result/*.wav')

In [ ]:
glob.glob('result/tone-*.wav')

In [ ]:
files = glob.glob('result/tone-*.wav')

for f in files:
    sr, samples = wavfile.read(f)
    samples = samples.astype(float)
    fade = linspace(1.0, 0, sr/2.0)
    samples[int(-sr/2):] *= fade
    wavfile.write(f.replace('.wav', '_fade.wav'), sr, samples.astype(int16))


In [ ]:
type(samples)

In [ ]:
sr, samples = wavfile.read('result/tone-440_fade.wav')
plot(samples)

In [ ]:
!afplay result/tone-440_fade.wav

Or on meta-data, using libraries like:
* https://code.google.com/p/mutagen/
* http://audiotools.sourceforge.net/

In [ ]:
from mutagen.easyid3 import EasyID3
audio = EasyID3("media/SoundHelix-Song-3.mp3")
audio["title"] = u"An example"
audio.save()

Why didn't this work? Try `pip3 install mutagen` on the terminal, then try the cell again.

In [ ]:
from mutagen.id3 import ID3, TIT2
audio = ID3("media/SoundHelix-Song-3.mp3")
audio.add(TIT2(encoding=3, text=u"An example"))
audio.save()

# Score/MIDI/Code generation

You can generate code or MIDI using python to drive other music production systems.

Generating text scores for languages like Csound is simple in Python thanks to it's string processing capabilities.

In [ ]:
# Csound score

score = ''
time = 0
dur = 0.5

import random

for i in range(10):
    freq = 440 + random.random() * 440 
    score += "i 1 %f %f %f\n"%(time, dur, freq)
    time += dur

In [ ]:
print(score)

A simple library for working with MIDI files:
* https://code.google.com/p/midiutil/

In [ ]:
from midiutil.MidiFile import MIDIFile
# Create the MIDIFile Object with 1 track
MyMIDI = MIDIFile(1)

# Tracks are numbered from zero. Times are measured in beats.

track = 0   
time = 0

# Add track name and tempo.
MyMIDI.addTrackName(track,time,"Sample Track")
MyMIDI.addTempo(track,time,120)

# Add a note. addNote expects the following information:
track = 0
channel = 0
pitch = 62
time = 0
duration = 1
volume = 100

# Now add the note.
MyMIDI.addNote(track,channel,pitch,time,duration,volume)

# And write it to disk.
binfile = open("result/output.mid", 'wb')
MyMIDI.writeFile(binfile)
binfile.close()

What should we do about this error? (Hint: we fixed something like this before.)

In [ ]:
!/usr/local/bin/fluidsynth -i media/gugs1.471.sf2 result/output.mid

Now generate a sequence of notes:

In [ ]:
from midiutil.MidiFile import MIDIFile
# Create the MIDIFile Object with 1 track
MyMIDI = MIDIFile(1)

track = 0   
time = 0

MyMIDI.addTrackName(track,time,"Sample Track")
MyMIDI.addTempo(track,time,120)

# Add a note. addNote expects the following information:
track = 0
channel = 0
pitch = 62
time = 0
duration = 1
volume = 100

for i in range(10):
    pitch = 48 + int(random.random() * 24)
    time += duration
    MyMIDI.addNote(track,channel,pitch,time,duration,volume)

# And write it to disk.
binfile = open("result/output_random.mid", 'wb')
MyMIDI.writeFile(binfile)
binfile.close()


In [ ]:
!/usr/local/bin/fluidsynth -i media/gugs1.471.sf2 result/output_random.mid

# Digital Signal Processing and Acoustics

Python can be useful as a prototyping tool for DSP and acoustics algorithms.

In [ ]:
from numpy import random

In [ ]:
sig = random.random(2048)*2 -1

In [ ]:
Pxx, freqs, times, im = specgram(sig, Fs=44100);
colorbar()
gcf().set_figwidth(16)

In [ ]:
times

In [ ]:
Pxx.shape

In [ ]:
plot(Pxx[:,10])

In [ ]:
filtered = (sig + r_[0,sig[:-1]])
Pxx2, freqs, times, im = specgram(filtered, Fs=44100);
colorbar()

In [ ]:
plot(Pxx2[:,10])

In [ ]:
plot(Pxx[:,10])
plot(Pxx2[:,10])

In [ ]:
plot(Pxx[:,10])
plot(Pxx2[:,10]/4.0)

http://docs.scipy.org/doc/scipy/reference/signal.html

In [ ]:
from scipy.signal import lfilter
filtered4 = lfilter([1],[1, 1], sig)
specgram(filtered4, Fs=44100);

##Biquad filters

$$H(z) = \frac{b_0z^{0} + b_1z^{-1} +  b_2z^{-2}}{ a_0z^{0} + a_1z^{-1} + a_2z^{-2}}$$

Great "cookbook":
[http://www.musicdsp.org/files/Audio-EQ-Cookbook.txt](http://www.musicdsp.org/files/Audio-EQ-Cookbook.txt)

In [ ]:
# low shelf-filter

Fs = 44100
f0 = 10000.0
dBgain = 30.0
S = 1.0 # shelf slope
# -----------------------
A  = 10**(dBgain/40)

w0 = 2*pi*f0/Fs
alpha = sin(w0)/2 * sqrt( (A + 1/A)*(1/S - 1) + 2 ) 
       
b0 =    A*( (A+1) - (A-1)*cos(w0) + 2*sqrt(A)*alpha )
b1 =  2*A*( (A-1) - (A+1)*cos(w0)                   )
b2 =    A*( (A+1) - (A-1)*cos(w0) - 2*sqrt(A)*alpha )
a0 =        (A+1) + (A-1)*cos(w0) + 2*sqrt(A)*alpha
a1 =   -2*( (A-1) + (A+1)*cos(w0)                   )
a2 =        (A+1) + (A-1)*cos(w0) - 2*sqrt(A)*alpha

In [ ]:
from scipy.signal import freqz
w, h = freqz([b0, b1, b2], [a0, a1, a2])
semilogy(w,abs(h))

In [ ]:
from scipy.signal import iirdesign

In [ ]:
Wp = 0.5  # Cutoff frequency 
Ws = 0.6   # Stop frequency 
Rp = 0.1     # passband maximum loss (gpass)
As = 60      # stoppand min attenuation (gstop)
b,a = iirdesign(Wp, Ws, Rp, As, ftype='butter')
frq, resp = freqz(b,a)
plot(frq, abs(resp))
twinx()
plot(frq, angle(resp), 'r')

title('Butterworth filter')
grid()

More details and examples:
* http://nbviewer.ipython.org/github/mantaraya36/201A-ipython/blob/master/Audio%20Filters.ipynb

# Sonification/Simulation/Big Data

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
index_0 = (iris.data[:,0] - iris.data[:,0].min())/ptp(iris.data[:,0])
index_1 = (iris.data[:,1] - iris.data[:,1].min())/ptp(iris.data[:,1])
space = zeros((512,512))

space[50 + (index_0*450).astype(int),50 + (index_1*450).astype(int)] += 1

imshow(space, cmap='gray', aspect='auto')
gcf().set_figheight(12)
gcf().set_figwidth(12)

You know what to do now, right?

In [ ]:
from scipy.ndimage.filters import gaussian_filter
gspace = gaussian_filter(space, 7)
imshow(gspace)

gcf().set_figheight(9)
gcf().set_figwidth(9)

In [ ]:
Ex,Ey = gradient(gspace)

n = 35000
dt = 1e-1

r = zeros((n,2))
v = zeros((n,2))
a = zeros((n,2))

r[0] = [200, 300]
v[0] = [1.0, -1.0]

damp = 0.9999

q = 5500
m = 0.5

for i in range(n - 1):
    E = array([Ex[int(r[i][0]), int(r[i][1])], Ey[int(r[i][0]), int(r[i][1])]])

    a[i+1] = q*E/m 
    v[i+1] = damp * (v[i] + a[i+1]*dt)
    r[i+1] = r[i] + v[i+1]*dt


In [ ]:
figure(figsize=(12,10))
imshow(gspace.T)
#quiver(Ex[::],Ey[::])
plot(*r.T, lw=3, c= 'w')
xlim((0, 512))
ylim((0, 512))

In [ ]:
vel = hypot(*v.T)
plot(hypot(*v.T))

In [ ]:
Ek = 0.5 * m * hypot(*v.T)**2
plot(Ek)

In [ ]:
import icsound
ics = icsound.icsound()
ics.start_engine()

# Music Information Retrieval

Various excellent libraries are available for research related to MIR in Python:
* http://essentia.upf.edu/
* https://github.com/bmcfee/librosa
* https://github.com/marsyas/marsyas

Generic machine learning:
* http://shogun-toolbox.org/page/documentation/information
* http://scikit-learn.org/stable/

http://nbviewer.ipython.org/github/mantaraya36/240E-ipython/tree/master/

# Synthesis

Python can be useful for prototyping synthesis techniques offline (to file), and there are synthesis libraries for Python like pyo:

* https://code.google.com/p/pyo/

In [ ]:
from pyo import *
s = Server().boot()

In [ ]:
a = Sine(440, 0, 0.1).out()
s.start()

In [ ]:
b = Sine(880, 0, 0.1).out()

In [ ]:
s.stop()

In [ ]:
s.shutdown()
s.boot()
b = Sine(880, 0, 0.1).out()
s.start()

In [ ]:
s.stop()

For some reason, the stop() function needs to settle before the shutdown(), so put them in separate cells...

In [ ]:
s.shutdown()
s.boot()

In [ ]:
s.shutdown()
s.boot()
s.start()
wav = SquareTable()
env = CosTable([(0,0), (100,1), (500,.3), (8191,0)])
met = Metro(.250, 12).play()
amp = TrigEnv(met, table=env, mul=.1)
pit = TrigXnoiseMidi(met, dist='loopseg', x1=20, scale=1, mrange=(48,84))
out = Osc(table=wav, freq=pit, mul=amp).out()

In [ ]:
s.stop()

In [ ]:
s.start()
wav = SquareTable()
env = CosTable([(0,0), (100,1), (500,.3), (8191,0)])
met = Metro(.250, 12).play()
amp = TrigEnv(met, table=env, mul=.1)
pit = TrigXnoiseMidi(met, dist='loopseg', x1=20, scale=1, mrange=(48,84))
out = Osc(table=wav, freq=pit, mul=amp).out()
s.gui(locals())

In [ ]:
%%csound
givalue = 10

In [ ]:
%%csound
print givalue

In [ ]:
ics.print_log()

In [ ]:
ics.fill_table(1, Ek)

In [ ]:
%%csound
chn_k "freq", 1

instr 1

kfreq chnget "freq"
asig poscil 1/45, kfreq, 1

outs asig, asig

endin

In [ ]:
ics.send_score("i 1 0 -1")

In [ ]:
ics.set_channel("freq", 1.0)

In [ ]:
ics.set_channel("freq", 0.5)

In [ ]:
ics.start_record("one_particle.wav")

In [ ]:
ics.stop_record()

In [ ]:
ics.send_score("i -1 0 -1")

In [ ]:
!aplay one_particle.wav

# By Andrés Cabrera mantaraya36@gmail.com

For course MAT 240A at UCSB

This ipython notebook is licensed under the CC-BY-NC-SA license: http://creativecommons.org/licenses/by-nc-sa/4.0/

![http://i.creativecommons.org/l/by-nc-sa/3.0/88x31.png](http://i.creativecommons.org/l/by-nc-sa/3.0/88x31.png)